# Imports

In [1]:
import torch
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from src.hyperparams_tuning import tune_params, final_train

In [2]:
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


# MovieLens 1M

In [3]:
max_epochs_ml = 3

## HyperParameter Tuning

In [4]:
n_trials = 2

study_ml, df_tuning_results_ml = tune_params(
    model_name='VAE', 
    dataset_name='movielens', 
    max_epochs=max_epochs_ml,
    n_trials=n_trials, 
    device=device
    )

[I 2022-02-06 11:24:45,960] A new study created in memory with name: no-name-1d8539ac-bce5-4d60-b5c7-d18e69cb1cad
/Users/guyattia/miniconda3/envs/RecSys/lib/python3.8/site-packages/optuna/distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [200, 600] which is of type list.
  warnings.warn(message)
/Users/guyattia/miniconda3/envs/RecSys/lib/python3.8/site-packages/optuna/distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [250, 500] which is of type list.
  warnings.warn(message)


Movie-lens file already exists
Training Results - Epoch[1] Avg loss: 40311.97
Validation Results - Epoch[1] Avg loss: 41154.94
Validation Results - Epoch[1] MRR: 0.63
Training Results - Epoch[2] Avg loss: 92207.63
Validation Results - Epoch[2] Avg loss: 93612.25
Validation Results - Epoch[2] MRR: 0.60
Training Results - Epoch[3] Avg loss: 43219.53
Validation Results - Epoch[3] Avg loss: 44095.81


[I 2022-02-06 11:25:57,616] Trial 0 finished with value: 41154.94009713977 and parameters: {'learning_rate': 0.01, 'activation_func': 'tanh', 'p_dims': [200, 600], 'optimizer': 'RMSprop', 'batch_size': 256}. Best is trial 0 with value: 41154.94009713977.


Validation Results - Epoch[3] MRR: 0.66
Movie-lens file already exists


/Users/guyattia/miniconda3/envs/RecSys/lib/python3.8/site-packages/optuna/distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [200, 600] which is of type list.
  warnings.warn(message)
/Users/guyattia/miniconda3/envs/RecSys/lib/python3.8/site-packages/optuna/distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [250, 500] which is of type list.
  warnings.warn(message)


Training Results - Epoch[1] Avg loss: 7840.82
Validation Results - Epoch[1] Avg loss: 8000.68
Validation Results - Epoch[1] MRR: 0.67
Training Results - Epoch[2] Avg loss: 7737.29
Validation Results - Epoch[2] Avg loss: 7892.21
Validation Results - Epoch[2] MRR: 0.66
Training Results - Epoch[3] Avg loss: 7607.45
Validation Results - Epoch[3] Avg loss: 7760.64


[I 2022-02-06 11:26:47,511] Trial 1 finished with value: 7760.643281165677 and parameters: {'learning_rate': 0.001, 'activation_func': 'selu', 'p_dims': [200, 600], 'optimizer': 'Adam', 'batch_size': 512}. Best is trial 1 with value: 7760.643281165677.


Validation Results - Epoch[3] MRR: 0.70


In [5]:
display(df_tuning_results_ml.sort_values(by='value').head(15))
best_params_ml = study_ml.best_params

,number,value,datetime_start,datetime_complete,duration,params_activation_func,params_batch_size,params_learning_rate,params_optimizer,params_p_dims,state
1,1,7760.643281,2022-02-06 11:25:57.617658,2022-02-06 11:26:47.510826,0 days 00:00:49.893168,selu,512,0.001,Adam,"[200, 600]",COMPLETE
0,0,41154.940097,2022-02-06 11:24:45.961672,2022-02-06 11:25:57.615825,0 days 00:01:11.654153,tanh,256,0.010,RMSprop,"[200, 600]",COMPLETE


## Full Training
- Use the best hyperparameters to build the final model
- Train the final model on the train+validation data sets (full_train)
- Test it against the test set for final results

In [7]:
test_loss_ml, final_model_ml, test_mrr_ml = final_train(
    model_name='VAE', 
    dataset_name='movielens', 
    max_epochs=max_epochs_ml,
    best_params=best_params_ml, 
    device=device
    )
print(f'Final test loss = {test_loss_ml}\nFinal test MRR = {test_mrr_ml}')

Movie-lens file already exists
Training Results - Epoch[1] Avg loss: 8037.25
Validation Results - Epoch[1] Avg loss: 8206.52
Validation Results - Epoch[1] MRR: 0.66
Training Results - Epoch[2] Avg loss: 7921.91
Validation Results - Epoch[2] Avg loss: 8081.56
Validation Results - Epoch[2] MRR: 0.64
Training Results - Epoch[3] Avg loss: 7779.43
Validation Results - Epoch[3] Avg loss: 7940.12
Validation Results - Epoch[3] MRR: 0.69
Final test loss = 7940.115488397194
Final test MRR = 0.6945059893722406


# Netflix

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=.